In [1]:
# NN Architecture: m'=4 Embedding: {7,32,64,64} Fitting:{256,128,1}

In [2]:
%matplotlib inline

# system modules
import os
import time
from pathlib import Path
from IPython.display import display, Math

# scientific computing
import numpy as np
from numpy import linalg as LA
import pandas as pd
np.random.seed(42)

# plotting
import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt
# from plotly.offline import iplot, init_notebook_mode
# import plotly.graph_objs as go
# import plotly.io as pio
# from plotly import subplots
# init_notebook_mode(connected=True)

# pytorch importing
import torch
import torch.nn as nn
from torchvision import datasets
from torch.optim import lr_scheduler, Adam
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
torch.manual_seed(42)

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(7,32)
        self.fc2 = nn.Linear(32,64)
        self.fc3 = nn.Linear(64,64)
        
        self.fc4 = nn.Linear(256,128)
        self.fc5 = nn.Linear(128,1)
        
        
    def forward(self, X):
        R1 = X           # Batchsize * stencil_size * n_features
        stencil = R1.shape[1]
        R2 = X[:,:,4:11] 
        G1 = self.relu(self.fc1(R2))
        G1 = self.relu(self.fc2(G1))
        G1 = self.fc3(G1)
        G2 = G1[:,:,0:4]
        
        G1t = G1.permute(0,2,1)
        R1t = R1.permute(0,2,1)
        D1 = torch.bmm(G1t,R1)/stencil
        D2 = torch.bmm(R1t,G2)/stencil
        D = torch.bmm(D1,D2)   # D = G1t*R*Rt*G2
        
        Dp = D.reshape(D.shape[0],-1)
        out = self.relu(self.fc4(Dp))
        out = self.fc5(out)
        
        return out

In [4]:
def train(train_loader, valid_loader, num_epoch):
    train_err_hist = torch.cuda.FloatTensor(1,1).fill_(0)
    valid_err_hist = torch.cuda.FloatTensor(1,1).fill_(0)
    train_loss_hist = torch.cuda.FloatTensor(1,1).fill_(0)
    valid_loss_hist = torch.cuda.FloatTensor(1,1).fill_(0)

    for epoch in range(num_epoch+1):
        train_loss_array = torch.cuda.FloatTensor(1,1).fill_(0)
        train_err_rate_num = torch.cuda.FloatTensor(1,1).fill_(0)
        train_err_rate_den = torch.cuda.FloatTensor(1,1).fill_(0)
        valid_loss_array = torch.cuda.FloatTensor(1,1).fill_(0)
        valid_err_rate_num = torch.cuda.FloatTensor(1,1).fill_(0)
        valid_err_rate_den = torch.cuda.FloatTensor(1,1).fill_(0)

        for i, data in enumerate(train_loader):
            features, target = data
            optimizer.zero_grad()
            forward = model(features)
            loss = loss_fn(forward, target)
            loss.backward()
            optimizer.step()

            train_loss_array = torch.cat((train_loss_array, torch.cuda.FloatTensor([[loss.item()]])))
            train_err_num, train_err_den = report_err_rate(target, forward)
            train_err_rate_num = torch.cat((train_err_rate_num, (train_err_num.view(1,-1))**2), 0)
            train_err_rate_den = torch.cat((train_err_rate_den, (train_err_den.view(1,-1))**2), 0)

        train_loss = torch.mean(train_loss_array)
        train_err_rate = 100*((torch.sum(train_err_rate_num, 0))**0.5)/((torch.sum(train_err_rate_den, 0))**0.5)

        exp_lr_scheduler.step()

        with torch.no_grad():
            for i, data_valid in enumerate(valid_loader):
                features_valid, target_valid = data_valid
                forward_valid = model(features_valid)
                pred_loss = loss_fn(forward_valid, target_valid)

                valid_loss_array = torch.cat((valid_loss_array, torch.cuda.FloatTensor([[loss.item()]])))
                valid_err_num, valid_err_den = report_err_rate(target_valid, forward_valid)
                valid_err_rate_num = torch.cat((valid_err_rate_num, (valid_err_num.view(1,-1))**2), 0)
                valid_err_rate_den = torch.cat((valid_err_rate_den, (valid_err_den.view(1,-1))**2), 0)

            valid_loss = torch.mean(valid_loss_array)
            valid_err_rate = 100*((torch.sum(valid_err_rate_num, 0))**0.5)/((torch.sum(valid_err_rate_den, 0))**0.5)

        verb = True if (epoch >= 50) and (epoch % 10 == 0) else False
        if (verb):
            train_loss_hist = torch.cat((train_loss_hist, torch.cuda.FloatTensor([[train_loss]])))
            train_err_hist = torch.cat((train_err_hist, train_err_rate.view(1,-1)), 0)
            valid_loss_hist = torch.cat((valid_loss_hist, torch.cuda.FloatTensor([[valid_loss]])))
            valid_err_hist = torch.cat((valid_err_hist, valid_err_rate.view(1,-1)), 0)
        verb = True if (epoch % 50 == 0) else False
        if (verb) :
            print('{:4}   lr: {:.2e}   train_loss: {:.2e}   valid_loss: {:.2e}   train_error:{:7.2f}%   valid_error:{:7.2f}%' \
                  .format(epoch, exp_lr_scheduler.get_lr()[0], train_loss, valid_loss, train_err_rate[0], valid_err_rate[0]))
            
    print('Finished Training')
    return train_loss_hist, train_err_hist, valid_loss_hist, valid_err_hist

In [5]:
def report_err_rate(target, forward):
    errRate_sigma_num = torch.norm(forward - target, dim = 0)
    errRate_sigma_den = torch.norm(target, dim = 0)
    return errRate_sigma_num, errRate_sigma_den

In [6]:
def error_plot(training_loss_history, training_error_history, valid_loss_history, valid_error_history):
    
    data1 = go.Scatter(x=np.arange(50,num_epoch+1,10), y=training_loss_history[1:,0], line = dict(width=1.7), name = 'Training Loss', mode = 'lines')
    data2 = go.Scatter(x=np.arange(50,num_epoch+1,10), y=valid_loss_history[1:,0], line = dict(width=1.7), name = 'Validation Loss', mode = 'lines')
    data3 = go.Scatter(x=np.arange(50,num_epoch+1,10), y=training_error_history[1:,0], line = dict(width=1.7), name = 'Training Error', mode = 'lines')
    data4 = go.Scatter(x=np.arange(50,num_epoch+1,10), y=valid_error_history[1:,0], line = dict(width=1.7), name = 'Validation Error', mode = 'lines')
    
    fig = subplots.make_subplots(rows=1, cols=2)
    fig.append_trace(data1, 1, 1)
    fig.append_trace(data2, 1, 1)
    fig.append_trace(data3, 1, 2)
    fig.append_trace(data4, 1, 2)
    
    fig['layout']['xaxis1'].update(title='Epochs', showgrid=True, gridwidth=0.5, gridcolor='lightgrey', showline=True, linecolor='black')
    fig['layout']['yaxis1'].update(title='Loss', showgrid=True, gridwidth=0.5, gridcolor='lightgrey', showline=True, linecolor='black')
    fig['layout']['xaxis2'].update(title='Epochs', showgrid=True, gridwidth=0.5, gridcolor='lightgrey', showline=True, linecolor='black')
    fig['layout']['yaxis2'].update(title='Error %', showgrid=True, gridwidth=0.5, gridcolor='lightgrey', showline=True, linecolor='black')
    fig['layout'].update(height=450, width=1000, plot_bgcolor = 'rgba(0,0,0,0)', title='Loss and Error Percentage History')
    iplot(fig)

In [7]:
if torch.cuda.is_available():
    device=torch.device('cuda:0')
else:
    device=torch.device('cpu')

device_cpu = torch.device('cpu')

In [8]:
stencil_size = 150

In [9]:
dataX_train = torch.load('new_dataX_train.pt')
dataY_train = torch.load('new_dataY_train.pt')
dataX_valid = torch.load('new_dataX_valid.pt')
dataY_valid = torch.load('new_dataY_valid.pt')

In [10]:
print('stencil size: {} \ntrain: Input:{}  Label:{} \nvalid: Input:{}  Label:{}' \
      .format(stencil_size,dataX_train.shape,dataY_train.shape,dataX_valid.shape,dataY_valid.shape))

stencil size: 150 
train: Input:torch.Size([1260000, 150, 11])  Label:torch.Size([1260000, 1]) 
valid: Input:torch.Size([20000, 150, 11])  Label:torch.Size([20000, 1])


In [11]:
dataX_train = dataX_train.to(device)
dataY_train = dataY_train.to(device)
dataX_valid = dataX_valid.to(device)
dataY_valid = dataY_valid.to(device)

In [12]:
#creating datasets
dataset_train = TensorDataset(dataX_train,dataY_train)
dataset_valid = TensorDataset(dataX_valid,dataY_valid)

#creating batches from dataset
batch_size_train = 1024        
batch_size_valid = dataX_valid.shape[0]

train_loader = DataLoader(dataset = dataset_train, batch_size=batch_size_train, shuffle=True)
valid_loader = DataLoader(dataset = dataset_valid, batch_size=batch_size_valid, shuffle=False)

In [13]:
np.random.seed(7)
model = Net()
model.to(device)
loss_fn = nn.MSELoss(reduction='sum')

In [14]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
para_count = count_parameters(model)
print('Total Learnable Parameters: {}'.format(para_count))

Total Learnable Parameters: 39553


In [15]:
# training
num_epoch = 2000
learning_rate = 1e-3
optimizer = Adam(model.parameters(), lr=learning_rate)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=600, gamma=0.7)

In [16]:
start_time = time.time()
training_loss_history, training_error_history, valid_loss_history, valid_error_history = train(train_loader, valid_loader, num_epoch)
elapsed = time.time() - start_time                
print('Training time: %.1f s' % (elapsed))

/home/xuhuizhou/miniconda3/envs/torch/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:369: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


   0   lr: 1.00e-03   train_loss: 3.24e+00   valid_loss: 1.56e-01   train_error:  13.27%   valid_error:   7.41%
  50   lr: 1.00e-03   train_loss: 3.04e-02   valid_loss: 6.88e-03   train_error:   1.29%   valid_error:   3.93%
 100   lr: 1.00e-03   train_loss: 1.68e-02   valid_loss: 4.31e-03   train_error:   0.96%   valid_error:   3.55%
 150   lr: 1.00e-03   train_loss: 1.22e-02   valid_loss: 2.89e-03   train_error:   0.81%   valid_error:   3.18%
 200   lr: 1.00e-03   train_loss: 9.66e-03   valid_loss: 2.25e-03   train_error:   0.72%   valid_error:   3.25%
 250   lr: 1.00e-03   train_loss: 8.17e-03   valid_loss: 2.73e-03   train_error:   0.67%   valid_error:   3.34%
 300   lr: 1.00e-03   train_loss: 7.18e-03   valid_loss: 1.54e-03   train_error:   0.63%   valid_error:   3.16%
 350   lr: 1.00e-03   train_loss: 6.57e-03   valid_loss: 1.48e-03   train_error:   0.60%   valid_error:   3.38%
 400   lr: 1.00e-03   train_loss: 6.06e-03   valid_loss: 1.21e-03   train_error:   0.57%   valid_error: 

In [17]:
# torch.save(model, 'test1_gpu.pt')
model.to(device_cpu)
torch.save(model, 'parametric_study_4_cpu.pt')